# Abstractive Text Summarization Using BART (bart-large-cnn)
_By Ling Li Ya_

References:
1. [Long Form Summarization With Hugging Face](https://github.com/nicknochnack/Longform-Summarization-with-Hugging-Face/blob/main/LongSummarization.ipynb)
2. [BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension](https://arxiv.org/abs/1910.13461)

## 1. Install and Import Dependencies
Install `pytorch`

In [1]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


You should consider upgrading via the 'C:\Users\liana\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Install `transformers` to use its `summarization pipeline`

In [2]:
!pip install transformers

You should consider upgrading via the 'C:\Users\liana\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Install `bs4` to use `BeautifulSoup`

In [3]:
!pip install bs4

You should consider upgrading via the 'C:\Users\liana\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Import all dependencies

In [4]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

## 2. Setup Generator
Get pipeline text summarization utility

In [5]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


## 3. Process Input Text
Get input from website URLs

In [6]:
URL = 'https://en.wikipedia.org/wiki/Rococo'

Get HTTP URL using `requests`

In [7]:
r = requests.get(URL)

Parse HTML body returned from the URL and format it to have a better readability

In [8]:
soup = BeautifulSoup(r.text, 'html.parser')

Find all text chunks with 'h1' and 'p' tags

In [9]:
results = soup.find_all(['h1', 'p'])
results[:3]

[<h1 class="firstHeading" id="firstHeading">Rococo</h1>,
 <p class="mw-empty-elt">
 </p>,
 <p><b>Rococo</b> (<span class="rt-commentedText nowrap"><span class="IPA nopopups noexcerpt"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="'r' in 'rye'">r</span><span title="/ə/: 'a' in 'about'">ə</span><span title="/ˈ/: primary stress follows">ˈ</span><span title="'k' in 'kind'">k</span><span title="/oʊ/: 'o' in 'code'">oʊ</span><span title="'k' in 'kind'">k</span><span title="/oʊ/: 'o' in 'code'">oʊ</span></span>/</a></span></span>, <small>also</small> <span class="rt-commentedText nowrap"><small><a href="/wiki/American_English" title="American English">US</a>: </small><span class="IPA nopopups noexcerpt"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="/ˌ/: secondary stress follows">ˌ</span><span title="'r' in 'rye'">r</span><span title="/oʊ/: 'o' in 'code'">oʊ</span><

Text enclosed within the HTML tags are selected and joined together

In [10]:
text = [result.text for result in results]
ARTICLE = ' '.join(text)
ARTICLE[0:1000]

"Rococo \n Rococo (/rəˈkoʊkoʊ/, also US: /ˌroʊkəˈkoʊ/), less commonly Roccoco or Late Baroque, is an exceptionally ornamental and theatrical style of architecture, art and decoration which combines asymmetry, scrolling curves, gilding, white and pastel colors, sculpted molding, and trompe-l'œil frescoes to create surprise and the illusion of motion and drama.  It is often described as the final expression of the Baroque movement.[1]\n The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV. It was known as the style rocaille, or rocaille style.[2] It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia.[3] It also came to influence the other arts, particularly sculpture, furniture, silverware, glassware, painting, music, and theatre.[4] Although originally a secular style primarily used for interiors of private residences the Rococo had a spiritual aspect to it w

## 4. Chunk text

Append <eos> to punctuations that marks the end of a sentence
<br />
Without the <eos> tag, sentences will be split without any punctuation

In [11]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
sentences = ARTICLE.split('<eos>')
sentences[:10]

["Rococo \n Rococo (/rəˈkoʊkoʊ/, also US: /ˌroʊkəˈkoʊ/), less commonly Roccoco or Late Baroque, is an exceptionally ornamental and theatrical style of architecture, art and decoration which combines asymmetry, scrolling curves, gilding, white and pastel colors, sculpted molding, and trompe-l'œil frescoes to create surprise and the illusion of motion and drama.",
 '  It is often described as the final expression of the Baroque movement.',
 '[1]\n The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV.',
 ' It was known as the style rocaille, or rocaille style.',
 '[2] It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia.',
 '[3] It also came to influence the other arts, particularly sculpture, furniture, silverware, glassware, painting, music, and theatre.',
 '[4] Although originally a secular style primarily used for interiors of private residences the Rococo

Limit the size of text in a chunk so that it is smaller than 500 words
<br />
Split sentences into words (2D array)
<br />
This is to avoid the error as shown below
```py
Token indices sequence length is longer than the specified maximum sequence length for this model (1024). Running this sequence through the model will result in indexing errors.
```

In [12]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        # Check if the chunk is less than 500 words
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        # Next chunk
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

print("A total of " + str(current_chunk + 1) + " chunks")
print("A total of " + str(len(chunks[0])) + " words in chunk[0]")

0
A total of 12 chunks
A total of 488 words in chunk[0]


Append words into sentences again where each chunk is ensured to have less than 500 words

In [13]:
for chunk_id in range (len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

print("A total of " + str(len(chunks[0].split(' '))) + " words in chunk[0]")

A total of 488 words in chunk[0]


## 5. Summarise Text

Summarise based on each chunk

In [14]:
res = summarizer(chunks,return_text="True")
res

[{'summary_text': ' The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV . It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia . The term rococo was first used in print in 1825 to describe decoration which was "out of style and old-fashioned"'},
 {'summary_text': ' Rocaille style, or French Rococo, appeared in Paris during the reign of Louis XV, and flourished between about 1723 and 1759 . Main ornaments of Rococo are: asymmetrical shells, acanthus and other leaves, birds, bouquets of flowers, fruits, musical instruments, angels and Chinoiserie (pagodas, dragons, monkeys, bizarre flowers and Chinese people) The style integrated painting, molded stucco, and wood carving, and quadratura, or illusionist ceiling paintings .'},
 {'summary_text': ' Rococo decorative style reached its summit in southern Germany and Austria from the 1730s until the 1770s . Notable

In [37]:
md = "" # Markdown output to be fed into slide generator
txt = ""
for result in res:
    for head, val in result.items():
        if(len(txt) == 0):
            md += "# " + head + "\n\n- " + val
            txt += val
        elif(len(txt) < 1000 and result != res[-1]): # If the number of characters on a slide exceeds 1000, create new slide
            md += "\n-" + val
            txt += val
        else:
            md += "\n\n---\n\n"
            txt = ""

print(md)
    

# summary_text

-  The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV . It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia . The term rococo was first used in print in 1825 to describe decoration which was "out of style and old-fashioned"
- Rocaille style, or French Rococo, appeared in Paris during the reign of Louis XV, and flourished between about 1723 and 1759 . Main ornaments of Rococo are: asymmetrical shells, acanthus and other leaves, birds, bouquets of flowers, fruits, musical instruments, angels and Chinoiserie (pagodas, dragons, monkeys, bizarre flowers and Chinese people) The style integrated painting, molded stucco, and wood carving, and quadratura, or illusionist ceiling paintings .
- Rococo decorative style reached its summit in southern Germany and Austria from the 1730s until the 1770s . Notable decorative painters included Giovanni Batt

## 6. Formatting Text
Preprocessing: format the `dict` object into a `string`.

In [ ]:
summary = ''
for result in res:
    summary += ''.join(str(val[1:]) + "\n" for _, val in result.items())

summary = summary.replace(' .', '.')
summary = summary.replace(" !", "!")
summary = summary.replace(" ?", "?")

## 7. Results

Some statistics and the final result.

In [ ]:
words_after = len(summary.split(' '))
words_before = len(ARTICLE)
reduced_by = (words_before - words_after) / words_before * 100

print("Number of words in summary: " + str(words_after))
print("Number of words in original article: " + str(words_before))
print("Reduced by: " + str(round(reduced_by, 2)) + "%\n")
print(summary)

Number of words in summary: 663
Number of words in original article: 37336
Reduced by: 98.22%

The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV. It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia. The term rococo was first used in print in 1825 to describe decoration which was "out of style and old-fashioned"
Rocaille style, or French Rococo, appeared in Paris during the reign of Louis XV. Main ornaments of Rococo are: asymmetrical shells, acanthus and other leaves, birds, bouquets of flowers, fruits, musical instruments, angels and Far Eastern elements. The style often integrated painting, molded stucco, and wood carving, and quadratura, or illusionist ceiling paintings.
Rococo decorative style reached its summit in southern Germany and Austria from the 1730s until the 1770s. Notable decorative painters included Giovanni Battista Tiepolo, who painted